# N0 configuration with lossy outer layer

Does this give us the waves we see numerically?  Is loss in material equivalent to PML?
We'll put loss in the material in same place and strength as PML, see if they give same results.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from fiberamp.fiber.microstruct.bragg import BraggExact
from step_exact import plotlogf
from ngsolve.webgui import Draw
from ngsolve import CF
from scipy.optimize import newton

# Initialize

In [ ]:
n = 300
wls = np.linspace(1.4e-6, 2e-6, n+1)
betas1 = np.zeros_like(wls, dtype=complex)
outer = 'h2'
nu = 1

In [ ]:
alpha=5
n_air = 1.00027717
n_glass = 1.4388164768221814
ts = [15*2.7183333333333333e-6, 15*2/3*1e-6, 15*1e-6]
ns = [lambda x: n_air, lambda x: n_glass, lambda x: n_glass - .03j]
mats = ['air', 'glass', 'Outer']
maxhs = [.1, .02, .04]
scale = 15e-6


In [ ]:
A = BraggExact(ts=ts, maxhs=[.1 ,.01, .01], wl=wls[0], ns=ns, mats=mats, scale=scale)


In [ ]:
Draw(CF(list(A.ns)),A.mesh)

In [ ]:
A.determinant(67, nu, outer)

In [ ]:
k_low = A.ks[0].real * A.scale
plotlogf(A.determinant, .999*k_low, 1.001 * k_low, -.015,.015, nu, outer,
         iref=100, rref=100, levels=100)

In [ ]:
guess = np.array(.99998*k_low)
imag = 0
x0 = np.array(guess + imag*1j)

beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
beta

In [ ]:
Fs = A.all_fields(beta, nu, outer)

In [ ]:
Draw(Fs['Ez'], A.mesh)

# Wavelength Loss Studies

## No loss base case

In [ ]:
ns = [lambda x: n_air, lambda x: n_glass, lambda x: n_glass]

for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, maxhs=[2,2,2], wl=wls[i], ns=ns, mats=mats, scale=scale)

    k_low = A.ks[0] * A.scale
    guess = np.array(.99999 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas1[i] = beta
    

In [ ]:
%matplotlib notebook
plt.figure(figsize=(8,5))
plt.plot(wls, -betas1.imag, color='green', linewidth=.9)
plt.yscale('log')


# A Bit of loss

In [ ]:
ns = [lambda x: n_air, lambda x: n_glass, lambda x: n_glass-.05j ]
betas2=np.zeros_like(betas1)

for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, maxhs=[2,2,2], wl=wls[i], ns=ns, mats=mats, scale=scale)

    k_low = A.ks[0] * A.scale
    guess = np.array(.99999 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas2[i] = beta
    

In [ ]:
%matplotlib notebook
plt.figure(figsize=(8,5))
plt.plot(wls, -betas1.imag, color='green', linewidth=.9, label='no loss')
plt.plot(wls, -betas2.imag, color='blue', linewidth=.9,label='.05j')
plt.yscale('log')
plt.legend()

# More loss

In [ ]:
ns = [lambda x: n_air, lambda x: n_glass, lambda x: n_glass-.1j ]
betas3=np.zeros_like(betas1)

for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, maxhs=[2,2,2], wl=wls[i], ns=ns, mats=mats, scale=scale)

    k_low = A.ks[0] * A.scale
    guess = np.array(.99999 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas3[i] = beta
    

In [ ]:
%matplotlib notebook
plt.figure(figsize=(8,5))
plt.plot(wls, -betas1.imag, color='green', linewidth=.9, label='no loss')
plt.plot(wls, -betas2.imag, color='blue', linewidth=.9,label='.05j')
plt.plot(wls, -betas3.imag, color='orange', linewidth=.9,label='0.1j')
plt.yscale('log')
plt.legend()

# More still : .2j

In [ ]:
ns = [lambda x: n_air, lambda x: n_glass, lambda x: n_glass-.2j ]
betas4=np.zeros_like(betas1)

for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, maxhs=[2,2,2], wl=wls[i], ns=ns, mats=mats, scale=scale)

    k_low = A.ks[0] * A.scale
    guess = np.array(.99999 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas4[i] = beta
    

In [ ]:
%matplotlib notebook
plt.figure(figsize=(10,6))
plt.plot(wls, -betas1.imag, color='green', linewidth=.9, label='no loss')
plt.plot(wls, -betas2.imag, color='blue', linewidth=.9,label='.05j')
plt.plot(wls, -betas3.imag, color='orange', linewidth=.9,label='0.1j')
plt.plot(wls, -betas4.imag, color='red', linewidth=.9,label='0.2j')
plt.yscale('log')
plt.legend()

# Load numerical data to compare


In [ ]:
num_betas = np.load('/home/pv/local/convergence/bragg_fiber/wavelength/N0/clean_betas_im.npy')

In [ ]:
%matplotlib notebook
plt.figure(figsize=(10,6))
plt.plot(wls, -betas1.imag, color='green', linewidth=.9, label='no loss')
plt.plot(wls, -betas2.imag, color='blue', linewidth=.9,label='.05j')
# plt.plot(wls, -betas3.imag, color='orange', linewidth=.9,label='0.1j')
# plt.plot(wls, -betas4.imag, color='red', linewidth=.9,label='0.2j')
plt.plot(wls, num_betas*A.scale, color='brown', linewidth=.9,linestyle='--',label='numerical')
plt.yscale('log')
plt.legend()
